In [2]:
import boto3
from botocore.client import Config
import os

In [3]:
# MinIO connection details
minio_endpoint = os.getenv('AWS_S3_ENDPOINT')
access_key = os.getenv('AWS_ACCESS_KEY_ID')
secret_key = os.getenv('AWS_SECRET_ACCESS_KEY')
bucket_name = "models"
region = os.getenv('AWS_DEFAULT_REGION')

# Initialize the S3 client
s3_client = boto3.client(
    's3',
    endpoint_url=minio_endpoint,
    aws_access_key_id=access_key,
    aws_secret_access_key=secret_key,
    config=Config(signature_version='s3v4')
)

# List objects in a specific bucket
objects = s3_client.list_objects_v2(Bucket=bucket_name)
if 'Contents' in objects:
    print("Objects found in bucket")
else:
    print("  No objects found.")

Objects found in bucket


In [11]:
experiment_name_param = os.getenv('experiment_name')
run_name_param = os.getenv('run_name')

In [21]:
model_to_score = f"fracture/{experiment_name_param}/Classif-Exp-{experiment_name_param}-Run-{run_name_param}.onnx"
current_directory = os.getcwd()
download_path_model_to_score = os.path.join(current_directory, "Classif-Exp-{experiment_name_param}-Run-{run_name_param}.onnx")

In [22]:
try:
    s3_client.download_file(bucket_name, model_to_score, download_path_model_to_score)
    print(f"Object downloaded successfully to '{download_path_model_to_score}'")
except Exception as e:
    print(f"Error occurred: {e}")

Object downloaded successfully to '/opt/app-root/src/Beroepsproduct/Models/Scoring models/Classif-Exp-{experiment_name_param}-Run-{run_name_param}.onnx'


In [27]:
download_folder = "production"

In [29]:
try:
    response = s3_client.list_objects_v2(Bucket=bucket_name, Prefix=download_folder)

    if 'Contents' in response:
        for obj in response['Contents']:
            object_name = obj['Key']

            base_name = os.path.basename(object_name)

            download_path = os.path.join(current_directory, base_name)

            s3_client.download_file(bucket_name, object_name, download_path)
            print(f"Object '{object_name}' downloaded successfully as '{download_path}'")
    else:
        print(f"No objects found in the folder '{download_folder}'")

except Exception as e:
    print(f"Error occurred: {e}")

Object 'production/Classif-Exp-fracture-classification-Run-run-2.onnx' downloaded successfully as '/opt/app-root/src/Beroepsproduct/Models/Scoring models/Classif-Exp-fracture-classification-Run-run-2.onnx'
